# Global Energy Grid Stress & Resilience Prediction

This notebook covers:

- Data preparation & checks
- Extensive EDA (ranges, missing values, target analysis)
- Feature importance analysis
- Model selection (linear + tree-based)
- Hyperparameter tuning and final model selection

**Target:** `grid_stress` (0 = normal, 1 = high stress)

The dataset is built from OPSD Time Series data using `src/download_data.py`.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
DATA_PATH = "../data/processed/energy_grid_daily.csv"

df = pd.read_csv(DATA_PATH)
df.head()


## Data quality checks

In [ ]:
df.info()


In [ ]:
df.isnull().sum()


In [ ]:
df.describe(include="all").T


## Target variable analysis

In [ ]:
target_dist = df["grid_stress"].value_counts().to_frame("count")
target_dist["share"] = target_dist["count"] / target_dist["count"].sum()
target_dist


In [ ]:
plt.figure(figsize=(4,3))
plt.bar(["0 (normal)", "1 (stress)"], df["grid_stress"].value_counts().sort_index().values)
plt.title("Target distribution")
plt.ylabel("count")
plt.show()


## Feature distributions

In [ ]:
num_cols = [c for c in df.columns if c not in ["country", "grid_stress"]]
df[num_cols].hist(bins=30, figsize=(12, 8))
plt.tight_layout()
plt.show()


## Country coverage

In [ ]:
df["country"].value_counts().head(20)


## Relationships and correlations

In [ ]:
corr = df[num_cols + ["grid_stress"]].corr()

plt.figure(figsize=(8,6))
plt.imshow(corr, aspect="auto")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.index)), corr.index)
plt.colorbar()
plt.title("Correlation matrix")
plt.tight_layout()
plt.show()


## Feature importance (tree-based baseline)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

X = df.drop(columns=["grid_stress"])
y = df["grid_stress"].astype(int)

cat_cols = ["country"]
num_cols = [c for c in X.columns if c not in cat_cols]

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", "passthrough", num_cols),
])

rf = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    class_weight="balanced",
    n_jobs=-1,
)

pipe = Pipeline([
    ("prep", preprocessor),
    ("model", rf),
])

pipe.fit(X, y)

# Extract feature names after one-hot encoding
ohe = pipe.named_steps["prep"].named_transformers_["cat"]
ohe_names = list(ohe.get_feature_names_out(cat_cols))
feature_names = ohe_names + num_cols

importances = pipe.named_steps["model"].feature_importances_
imp = pd.Series(importances, index=feature_names).sort_values(ascending=False)

imp.head(15)


In [ ]:
plt.figure(figsize=(8,5))
imp.head(15).sort_values().plot(kind="barh")
plt.title("Top 15 feature importances (Random Forest)")
plt.xlabel("importance")
plt.tight_layout()
plt.show()


## Model selection and tuning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 1) Logistic Regression baseline
logreg = Pipeline([
    ("prep", preprocessor),
    ("model", LogisticRegression(max_iter=2000, class_weight="balanced", random_state=42)),
])
logreg.fit(X_train, y_train)
auc_log = roc_auc_score(y_val, logreg.predict_proba(X_val)[:, 1])
auc_log


In [ ]:
# 2) Random Forest tuned
rf_pipe = Pipeline([
    ("prep", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=400,
        random_state=42,
        class_weight="balanced",
        n_jobs=-1
    )),
])

rf_params = {
    "model__max_depth": [3, 5, 8, 12, None],
    "model__min_samples_leaf": [1, 2, 4, 8],
    "model__min_samples_split": [2, 5, 10],
    "model__max_features": ["sqrt", "log2", None],
}

rf_search = RandomizedSearchCV(
    rf_pipe,
    rf_params,
    n_iter=15,
    scoring="roc_auc",
    cv=3,
    random_state=42,
    n_jobs=-1
)

rf_search.fit(X_train, y_train)
rf_best = rf_search.best_estimator_
auc_rf = roc_auc_score(y_val, rf_best.predict_proba(X_val)[:, 1])

auc_rf, rf_search.best_params_


In [ ]:
# 3) XGBoost tuned
pos = y_train.sum()
neg = len(y_train) - pos
scale_pos_weight = float(neg / max(pos, 1))

xgb_pipe = Pipeline([
    ("prep", preprocessor),
    ("model", XGBClassifier(
        n_estimators=600,
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )),
])

xgb_params = {
    "model__max_depth": [3, 4, 5, 6],
    "model__learning_rate": [0.02, 0.05, 0.1],
    "model__subsample": [0.7, 0.85, 1.0],
    "model__colsample_bytree": [0.7, 0.85, 1.0],
    "model__min_child_weight": [1, 3, 5],
    "model__reg_lambda": [0.0, 1.0, 5.0],
    "model__scale_pos_weight": [scale_pos_weight],
}

xgb_search = RandomizedSearchCV(
    xgb_pipe,
    xgb_params,
    n_iter=18,
    scoring="roc_auc",
    cv=3,
    random_state=42,
    n_jobs=-1
)

xgb_search.fit(X_train, y_train)
xgb_best = xgb_search.best_estimator_
auc_xgb = roc_auc_score(y_val, xgb_best.predict_proba(X_val)[:, 1])

auc_xgb, xgb_search.best_params_


In [ ]:
results = pd.DataFrame([
    {"model": "logreg", "roc_auc": auc_log},
    {"model": "rf_tuned", "roc_auc": auc_rf},
    {"model": "xgb_tuned", "roc_auc": auc_xgb},
]).sort_values("roc_auc", ascending=False)

results


## Conclusions

- The dataset shows clear differences between normal vs stress days in demand peak ratio and capacity margin proxy.
- Tree-based models outperform the linear baseline in most configurations.
- Hyperparameter tuning improves ROC-AUC and provides a transparent selection process.
- The final training and model export for production are implemented in `src/train.py`.
